In [11]:
from elasticsearch import Elasticsearch

In [12]:
from dotenv import load_dotenv
load_dotenv(override=True)
import os 
import json

es = Elasticsearch(
    os.getenv("ES_URL"), 
    basic_auth=(os.getenv("ES_USERAME"), os.getenv("ES_PASSWORD"))
)
print(os.getenv("ES_URL"))

search_str = "6825"

body_params = {
    "size": 50,
    "sort": [{"@timestamp": "desc"}],
    "query": {
        "bool": {
            "should": [
                {
                    "wildcard": {
                        "sessionId": { "value": "*{}*".format(search_str) }
                    }
                },
                {
                    "wildcard": {
                        "cartId": { "value": "*{}*".format(search_str) }
                    }
                }
            ]
        }
    }
}

res = es.search(index=os.getenv("ES_INDEX"), body=json.dumps(body_params))

print(res['hits']['hits'])

http://localhost:9200
[{'_index': 'test-logs', '_id': 'QsWxwJcBISrdcGFvmVDG', '_score': None, '_source': {'@timestamp': '2025-06-08T11:04:10Z', 'level': 'INFO', 'service': 'cart-service', 'message': 'Session started', 'cartId': 'CART33', 'sessionId': 'SESS6825'}, 'sort': [1749380650000]}, {'_index': 'test-logs', '_id': 'QcWxwJcBISrdcGFvmVCN', '_score': None, '_source': {'@timestamp': '2025-06-08T11:03:15Z', 'level': 'INFO', 'service': 'order-service', 'message': 'Order placed', 'cartId': 'CART33', 'sessionId': 'SESS06825', 'orderNum': 'ORD795'}, 'sort': [1749380595000]}, {'_index': 'test-logs', '_id': 'QMWxwJcBISrdcGFvmVBh', '_score': None, '_source': {'@timestamp': '2025-06-08T11:02:00Z', 'level': 'INFO', 'service': 'cart-service', 'message': 'Item added to cart', 'cartId': 'CART33', 'sessionId': 'SESS06825', 'item': 'mob002'}, 'sort': [1749380520000]}, {'_index': 'test-logs', '_id': 'P8WxwJcBISrdcGFvmVAr', '_score': None, '_source': {'@timestamp': '2025-06-08T11:01:05Z', 'level': 'IN

In [43]:
examples = [
    {
        "question": "Show me the cart details for Cart123",
        "query": """{{ "query": {{ "match": {{ "cartId": "Cart123" }} }} }}"""
    },
    {
        "question": "What are the orders placed between June 20 to June 25 2025?",
        "query": """{{ "query": {{ "bool": {{ "must": [
            {{ "range": {{ "@timestamp": {{ "gte": "2025-06-20T00:00:00Z", "lte": "2025-06-25T23:59:59Z" }} }} }},
            {{ "exists": {{ "field": "orderNum" }} }}
        ] }} }} }}"""
    }
]

In [14]:
from langchain_core.prompts import PromptTemplate

# user_prompt = PromptTemplate.from_template("Question: {question} \n ES Query: {query}")
example_prompt = PromptTemplate(
    input_variables=["question", "query"],
    template="Question: {question}\nES Query: {query}"
)

In [15]:
# Test the prompt formatting
example_prompt.invoke(examples[0])

StringPromptValue(text='Question: Show me the cart details for Cart123\nES Query: {{ "query": {{ "match": {{ "cartId": "Cart123" }} }} }}')

In [58]:
from langchain_core.prompts import FewShotPromptTemplate 

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are an Elastisearch DSL generator. In the output, display only the ES query in a valid JSON format. Do not include any additional text or explanations.\n",
    suffix="User input: {question}\n ES Query:",
    input_variables=["question"]
)
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'Show me the cart details for Cart123', 'query': '{{ "query": {{ "match": {{ "cartId": "Cart123" }} }} }}'}, {'question': 'What are the orders placed between June 20 to June 25 2025?', 'query': '{{ "query": {{ "bool": {{ "must": [\n            {{ "range": {{ "@timestamp": {{ "gte": "2025-06-20T00:00:00Z", "lte": "2025-06-25T23:59:59Z" }} }} }},\n            {{ "exists": {{ "field": "orderNum" }} }}\n        ] }} }} }}'}], example_prompt=PromptTemplate(input_variables=['query', 'question'], input_types={}, partial_variables={}, template='Question: {question}\nES Query: {query}'), suffix='User input: {question}\n ES Query:', prefix='You are an Elastisearch DSL generator. In the output, display only the ES query in a valid JSON format. Do not include any additional text or explanations.\n')

In [17]:
# Debug code
print("Example keys:")
for i, ex in enumerate(prompt.examples):
    print(f"Example {i}:", list(ex.keys()))

Example keys:
Example 0: ['question', 'query']
Example 1: ['question', 'query']


In [59]:
final_prompt = prompt.format(
    question="Show me the details of session id SESS-123"
    # question="Show me the details of order no ORD-5787"
    # question="List all sessions created on 1st July 2025" -> incorrect

)
final_prompt

'You are an Elastisearch DSL generator. In the output, display only the ES query in a valid JSON format. Do not include any additional text or explanations.\n\n\nQuestion: Show me the cart details for Cart123\nES Query: { "query": { "match": { "cartId": "Cart123" } } }\n\nQuestion: What are the orders placed between June 20 to June 25 2025?\nES Query: { "query": { "bool": { "must": [\n            { "range": { "@timestamp": { "gte": "2025-06-20T00:00:00Z", "lte": "2025-06-25T23:59:59Z" } } },\n            { "exists": { "field": "orderNum" } }\n        ] } } }\n\nUser input: Show me the details of session id SESS-123\n ES Query:'

In [48]:
from langchain_groq import ChatGroq 

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="Llama3-8b-8192"
)

In [60]:
# Get LLM output 
response = llm.invoke(final_prompt)

response

AIMessage(content='{ "query": { "match": { "sessionId": "SESS-123" } } }', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 200, 'total_tokens': 222, 'completion_time': 0.017830957, 'prompt_time': 0.023242829, 'queue_time': 0.27051968, 'total_time': 0.041073786}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_2717d04279', 'finish_reason': 'stop', 'logprobs': None}, id='run--608e568d-5e9a-4eac-8601-3d9cd70de32e-0', usage_metadata={'input_tokens': 200, 'output_tokens': 22, 'total_tokens': 222})

In [56]:
# Helper functions

import json 

def extract_json(text):
    text = text.replace("\n", "").replace(" ", "").strip()
    try:
        # Try direct parsing
        return json.loads(text)
    except json.JSONDecodeError:
        # Extract substring between curly braces {} 
        try:
            json_part = text[text.index('{'):text.rindex('}')+1]
            print("JSON part extracted:", json_part)
            return json.loads(json_part)
        except Exception as e:
            print("Failed to extract JSON:", e)
            return None
        
def is_valid_es_query(es_obj):
    if not isinstance(es_obj, dict):
        return False 
    return "query" in es_obj

In [62]:
# Get ES query from AI response

es_query = extract_json(response.content)

if es_query and is_valid_es_query(es_query):
    print("Valid Elasticsearch query:", es_query)
else: 
    print("Invalid Elasticsearch query.")

Valid Elasticsearch query: {'query': {'match': {'sessionId': 'SESS-123'}}}


In [64]:
# Pass the parsed ES query to Elasticsearch

response = es.search(index=os.getenv("ES_INDEX"), body=json.dumps(es_query))
# print(res['hits']['hits'])
hits = response.get("hits", {}).get("hits", [])

logs = []
for i,doc in enumerate(hits, 1):
    source = doc['_source']
    logs.append({
        "message": source.get("message", "-"), 
        "@timestamp": source.get("@timestamp", "-"), 
        "sessionId": source.get("sessionId", "-"),
        "cartId": source.get("cartId", "-"), 
        "orderNum": source.get("orderNum", "-")  
    })
    print(f"{i}. {source}")

print(logs)


1. {'@timestamp': '2025-06-26T17:25:20Z', 'level': 'INFO', 'service': 'cart-service', 'message': 'Item added to cart', 'cartId': 'CART-A1F7', 'sessionId': 'SESS-9X3Z', 'item': 'item-7B3D'}
2. {'@timestamp': '2025-06-26T17:26:10Z', 'level': 'INFO', 'service': 'order-service', 'message': 'Order placed', 'cartId': 'CART-A1F7', 'sessionId': 'SESS-9X3Z', 'orderNum': 'ORD-91AZ'}
3. {'@timestamp': '2025-06-26T17:27:00Z', 'level': 'INFO', 'service': 'cart-service', 'message': 'Session started', 'cartId': 'CART-A1F7', 'sessionId': 'SESS-2LT9'}
4. {'@timestamp': '2025-06-06T17:28:30Z', 'level': 'INFO', 'service': 'order-service', 'message': 'Order placed', 'cartId': 'CART-A1F7', 'sessionId': 'SESS-2LT9', 'orderNum': 'ORD-ZY19'}
5. {'@timestamp': '2025-06-26T17:25:03Z', 'level': 'INFO', 'service': 'cart-service', 'message': 'Session started', 'cartId': 'CART-A1F7', 'sessionId': 'SESS-9X3Z'}
[{'message': 'Item added to cart', '@timestamp': '2025-06-26T17:25:20Z', 'sessionId': 'SESS-9X3Z', 'cartId'